In [6]:
import pandas as pd



In [7]:
# Load the main big file
df = pd.read_csv('transactions.csv')
print(f"Total Rows: {len(df)}")



Total Rows: 2266100


In [8]:
# View first few rows
df.head()


,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30


In [9]:
# Keep only rows where Days Before Departure == 15
df_15 = df[df['dbd'] == 15]




In [10]:
# Check how many records we got
print(f"Filtered Rows with dbd=15: {len(df_15)}")
df_15.head()

Filtered Rows with dbd=15: 73100


,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
1500,2023-03-01,2023-02-14,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,16.0,480.0,15
1501,2023-03-01,2023-02-14,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,34.0,352.0,15
1502,2023-03-01,2023-02-14,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,36.0,892.0,15
1503,2023-03-01,2023-02-14,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,18.0,1130.0,15
1504,2023-03-01,2023-02-14,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,48.0,1023.0,15


In [11]:
df_15.to_csv('filtered_transactions_15.csv', index=False)
files.download('filtered_transactions_15.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
train = pd.read_csv('train.csv')
filtered_txn = pd.read_csv('filtered_transactions_15.csv')

print("Train:", train.shape)
print("Filtered transactions:", filtered_txn.shape)

train.head()


Train: (67200, 4)
Filtered transactions: (73100, 11)


,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0


In [14]:
merged = pd.merge(train, filtered_txn, on=['doj', 'srcid', 'destid'], how='left')
print("Merged shape:", merged.shape)
merged.head()


Merged shape: (67200, 12)


,doj,srcid,destid,final_seatcount,doi,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,45,46,2838.0,2023-02-14,Karnataka,Tamil Nadu,Tier 1,Tier 1,16.0,480.0,15
1,2023-03-01,46,45,2298.0,2023-02-14,Tamil Nadu,Karnataka,Tier 1,Tier 1,34.0,352.0,15
2,2023-03-01,45,47,2720.0,2023-02-14,Karnataka,Andhra Pradesh,Tier 1,Tier 1,36.0,892.0,15
3,2023-03-01,47,45,2580.0,2023-02-14,Andhra Pradesh,Karnataka,Tier 1,Tier 1,18.0,1130.0,15
4,2023-03-01,46,9,4185.0,2023-02-14,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,48.0,1023.0,15


In [16]:
# Copy merged to avoid accidental overwrite
df = merged.copy()

# Fill missing values (if any)
df['cumsum_seatcount'] = df['cumsum_seatcount'].fillna(0)
df['cumsum_searchcount'] = df['cumsum_searchcount'].fillna(0)

# Convert 'doj' to datetime
df['doj'] = pd.to_datetime(df['doj'])

# Extract date features
df['journey_day'] = df['doj'].dt.day
df['journey_month'] = df['doj'].dt.month
df['journey_weekday'] = df['doj'].dt.weekday  # Monday = 0, Sunday = 6

# Encode categorical variables (region + tier)
for col in ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']:
    df[col] = df[col].astype('category').cat.codes

# Drop columns we don't need
df = df.drop(columns=['doj'])

# Display final features
df.head()


,srcid,destid,final_seatcount,doi,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd,journey_day,journey_month,journey_weekday
0,45,46,2838.0,2023-02-14,3,9,0,0,16.0,480.0,15,1,3,2
1,46,45,2298.0,2023-02-14,9,3,0,0,34.0,352.0,15,1,3,2
2,45,47,2720.0,2023-02-14,3,0,0,0,36.0,892.0,15,1,3,2
3,47,45,2580.0,2023-02-14,0,3,0,0,18.0,1130.0,15,1,3,2
4,46,9,4185.0,2023-02-14,9,9,0,3,48.0,1023.0,15,1,3,2


In [48]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd # Ensure pandas is imported

# List of columns to encode
cat_cols = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']

# Fit encoders on the combined training and testing data
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    # Convert columns to string type before concatenation
    combined_data = pd.concat([X[col].astype(str), test_merged[col].astype(str)], axis=0)
    le.fit(combined_data)

    # Transform the training and testing data separately
    X[col] = le.transform(X[col].astype(str))
    test_merged[col] = le.transform(test_merged[col].astype(str))

    encoders[col] = le # optional: save encoders

# Check all column types after encoding
print(X.dtypes)

srcid                   int64
destid                  int64
srcid_region            int64
destid_region           int64
srcid_tier              int64
destid_tier             int64
cumsum_seatcount      float64
cumsum_searchcount    float64
dbd                     int64
journey_day             int32
journey_month           int32
journey_weekday         int32
dtype: object


In [49]:
# Check all column types
print(X.dtypes)




srcid                   int64
destid                  int64
srcid_region            int64
destid_region           int64
srcid_tier              int64
destid_tier             int64
cumsum_seatcount      float64
cumsum_searchcount    float64
dbd                     int64
journey_day             int32
journey_month           int32
journey_weekday         int32
dtype: object


In [33]:
test = pd.read_csv('/content/test_8gqdJqH.csv')


In [51]:
# Check all column types
print(X.dtypes)

# Remove the line below as 'doi' does not exist and 'doj' was already dropped
# X = X.drop(columns=['doi'])

# X_train and y_train should be defined here before being used by train_test_split
# Assuming X is the feature DataFrame and y is the target variable DataFrame/Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np # Make sure numpy is imported

# Define y before splitting, assuming 'price' is the target variable
# You need to define y based on your dataset. For example:
# y = df['price']
# X = df.drop(columns=['price']) # X should be the features

# Based on the traceback, X already exists and has 12 columns.
# You need to ensure y is defined correctly from your original merged DataFrame before dropping target column from X.
# For demonstration, let's assume X is already prepared as intended and y needs to be defined.
# Assuming the target variable is 'price' in the original 'merged' or 'df' DataFrame
# You need to make sure you have saved the target variable before dropping it or transforming the DataFrame X

# Placeholder for defining y - YOU NEED TO replace this with how y is actually defined
# Example: y = merged['target_column_name']
# Since 'merged' and 'df' are available in the global scope, and 'X' is derived from 'df' after transformations,
# you should define 'y' from 'df' *before* dropping columns that might be the target variable.
# For instance, if your target is 'seat_count', you would do:
# y = df['seat_count']
# X = df.drop(columns=['seat_count']) # assuming 'seat_count' is the target

# Re-running the previous data preparation steps to ensure X and y are correctly defined
# based on the complete code snippet provided in the traceback context:

# Assuming the target is 'seat_count' or similar and was part of the merged dataframe.
# Let's assume you need to split the 'df' DataFrame from the previous step into features (X) and target (y).

# Redefine df from merged to ensure we have the correct columns for splitting
# (This assumes the notebook was executed sequentially and df contains all necessary columns including the target)
df = merged.copy()

# Fill missing values (if any)
df['cumsum_seatcount'] = df['cumsum_seatcount'].fillna(0)
df['cumsum_searchcount'] = df['cumsum_searchcount'].fillna(0)

# Convert 'doj' to datetime
df['doj'] = pd.to_datetime(df['doj'])

# Extract date features
df['journey_day'] = df['doj'].dt.day
df['journey_month'] = df['doj'].dt.month
df['journey_weekday'] = df['doj'].dt.weekday  # Monday = 0, Sunday = 6

# Encode categorical variables (region + tier)
for col in ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']:
    df[col] = df[col].astype('category').cat.codes

# At this point, decide which column is your target variable (y).
# Assuming 'seat_count' (from the original transactions data) is the target you want to predict.
# You need to ensure 'seat_count' was included in the 'filtered_txn' and subsequently in 'merged' and 'df'.
# If 'seat_count' is present in 'df', define y and X:

# --- Assuming 'seat_count' is the target variable ---
# Check if 'seat_count' is in df's columns
if 'seat_count' in df.columns:
    y = df['seat_count']
    X = df.drop(columns=['seat_count', 'doj']) # Drop target and original date column
else:
    print("Error: 'seat_count' column not found in DataFrame. Please check your data sources and merge process.")
    # You might need to adjust this based on your actual target column name
    # For example, if your target is 'search_count', use that instead.
    # raise KeyError("'seat_count' not found. Cannot define target variable.") # Or raise an error

# Ensure the categorical columns are correctly encoded here *before* splitting,
# using the LabelEncoder approach from your ipython-input-48 cell if needed,
# or rely on the .cat.codes from the previous step if that's sufficient.
# The .cat.codes approach is simpler and already done in the cell creating df.
# Let's stick with the .cat.codes encoding already applied to df.

# Ensure X and y are defined before proceeding to train_test_split

# --- Begin original train_test_split and model training code ---
# This block assumes X and y are correctly defined by the code above this line

# Check the shapes before splitting
if 'X' in locals() and 'y' in locals() and X is not None and y is not None:
    print("Shape of X:", X.shape)
    print("Shape of y:", y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE:", rmse)
else:
    print("X or y is not defined. Skipping model training.")

# --- End original train_test_split and model training code ---

srcid                   int64
destid                  int64
srcid_region            int64
destid_region           int64
srcid_tier              int64
destid_tier             int64
cumsum_seatcount      float64
cumsum_searchcount    float64
dbd                     int64
journey_day             int32
journey_month           int32
journey_weekday         int32
dtype: object
Error: 'seat_count' column not found in DataFrame. Please check your data sources and merge process.
Shape of X: (67200, 12)
Shape of y: (67200,)
RMSE: 471.9717240126706


In [52]:
test = pd.read_csv('/content/test_8gqdJqH.csv')  # Load test file again if needed

# Merge with transactions (make sure transactions are already defined)
test_merged = pd.merge(test, transactions, on=['doj', 'srcid', 'destid'], how='left')

# Fill NaNs if necessary
test_merged['cumsum_seatcount'] = test_merged['cumsum_seatcount'].fillna(0)
test_merged['cumsum_searchcount'] = test_merged['cumsum_searchcount'].fillna(0)

# Convert 'doj' to datetime
test_merged['doj'] = pd.to_datetime(test_merged['doj'])

# Extract date parts
test_merged['journey_day'] = test_merged['doj'].dt.day
test_merged['journey_month'] = test_merged['doj'].dt.month
test_merged['journey_weekday'] = test_merged['doj'].dt.weekday


In [53]:
for col in ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']:
    test_merged[col] = test_merged[col].astype('category').cat.codes


In [54]:
X_test_final = test_merged.drop(columns=['doj', 'doi', 'route_key'])  # Keep only features


In [55]:
test_preds = model.predict(X_test_final)

submission = pd.DataFrame({
    'route_key': test_merged['route_key'],
    'final_seatcount': test_preds
})

submission.to_csv('submission.csv', index=False)
print("✅ Submission file 'submission.csv' created!")


✅ Submission file 'submission.csv' created!
